# $G \cup n \; \sqrt{-1} \; \ell \; e \; \emptyset \quad \cdot \quad$ **Project Notebook**

---

<h2 align="center"><b>Table of Contents</b></h2>

1. [Code structure](#1-code-structure)
2. [Import of the Packages](#2-import-of-the-packages)
3. [Data Loading](#3-data-loading)
4. [Model Settings](#4-model-settings)

$$
\newcommand{\goto}{\; \longrightarrow \;}
\newcommand{\tdconv}{\text{2D Convolution} }
\newcommand{\relu}{\text{ReLU} }
$$

---

## 1) Code Structure

The code structure is the following:

```python
project
 ├ assets
 │  ├ cnn.py
 │  └ dataloader.py
 └ data
```

---

## 2) Import of the Packages

Standard packages needed that can be installed with either `conda` or `pip`:

In [8]:
# Pytorch imports
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn
import torchmetrics

# Utils imports
import numpy as np
import matplotlib.pyplot as plt
import pandas

Custom imports from our libraries:

In [11]:
from assets.dataloader import DataLoader
from assets.cnn import LabialCNN

---

## 3) Data Loading

In [12]:
# Create the dataloaders of our project

---

## 4) Model Settings

The following settings are applied:
> `device`: specifies where the model must be trained. If an Nvidia GPU is detected, then CUDA will be used;<br>
> `epochs`: the number of epochs;<br>
> `batch_size`: the size of each singular batch of analysed images;<br>
> `learning_rate`: `N/A`;<br>
> `loss_fn`: the loss function of the model;<br>
> `optimizer`: the optimizer of the model. For now it's `AdamW`, which is more performant than `SGD`.

The model has the following layers:

$$
\underbrace{x}_{\text{input}} \goto \underbrace{z_0(1, \; 5, \; 3)}_{\tdconv} \goto \underbrace{z_0(1, \; 5, \; 3)}_{\tdconv}
$$

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = LabialCNN().to(device)

epochs = 2
batch_size = 16
learning_rate = 0.0001

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)